In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import cv2
# Load the image
img = cv2.imread('model/test_images/maria-sharapova1.jpg')

print(img.shape)

In [3]:
plt.imshow(img)

In [4]:
gray=cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
gray.shape

In [5]:
gray

In [6]:
plt.imshow(gray , cmap='gray')

In [7]:
face_cascade=cv2.CascadeClassifier('model/opencv/haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier('model/opencv/haarcascade_eye.xml')

faces=face_cascade.detectMultiScale(gray , 1.3 , 5)
faces


In [8]:
(x,y,w,h)=faces[0]
x,y,w,h

In [9]:
face_img=cv2.rectangle(img,(x,y),(x+w , y+h),(255,0,0),2)
plt.imshow(face_img)

In [10]:
cv2.destroyAllWindows()
for(x,y,w,h) in faces:
    face_img=cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray=gray[y:y+h , x:x+w]
    roi_color=face_img[y:y+h , x:x+w]
    eyes=eye_cascade.detectMultiScale(roi_gray)
    for(ex , ey , ew , eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew , ey+eh),(0,255,0),2)
        

plt.figure()
plt.imshow(face_img , cmap='gray')
plt.show()

In [11]:
%matplotlib inline
plt.imshow(roi_color , cmap='gray')

In [12]:
cropped_img=np.array(roi_color)
cropped_img.shape

In [13]:
import numpy as np
import pywt as py
import cv2

In [14]:
def w2d(img , mode="haar" , level=1):
    imArray=img
    
    #Datatype conversions
    #Convert to grayscale
    imArray=cv2.cvtColor(imArray , cv2.COLOR_RGB2GRAY)
    
    #Convert to float
    imArray=np.float32(imArray)
    imArray /=255;
    
    #Compute coefficients
    coeffs=py.wavedec2(imArray , mode , level=level)
    
    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0]*=0;
    
    #Reconstruction
    imArray_H=py.waverec2(coeffs_H , mode);
    imArray_H *=255;
    imArray_H = np.uint8(imArray_H)
    
    return imArray_H

In [15]:
im_har=w2d(cropped_img , 'db1' , 5)
plt.imshow(im_har , cmap='gray')

In [16]:
def get_cropped_image_if_2_eyes(image_path):
    img=cv2.imread(image_path)
    gray=cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray , 1.3 , 5)
    for(x,y,w,h) in faces:
        roi_gray=gray[y:y+h , x:x+w]
        roi_color=img[y:y+h , x:x+w]
        eyes=eye_cascade.detectMultiScale(roi_gray)
        if len(eyes)>=2:
            return roi_color

In [17]:
original_image=cv2.imread('model/test_images/maria-sharapova1.jpg')
plt.imshow(original_image)

In [18]:
cropped_image=get_cropped_image_if_2_eyes('model/test_images/maria-sharapova1.jpg')
plt.imshow(cropped_image)

In [19]:
org_image_obstructed = cv2.imread('model/test_images/maria-sharapova2.jpg')
plt.imshow(org_image_obstructed)


In [20]:
cropped_image_no_2_eyes=get_cropped_image_if_2_eyes('model/test_images/maria-sharapova2.jpg')
cropped_image_no_2_eyes

In [21]:
path_to_data='model/dataset/'
path_to_cr_data='model/dataset/cropped/'

In [22]:
import os

# List to store the paths of all subdirectories
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)




In [23]:
img_dirs

In [24]:
import shutil
if os.path.exists(path_to_cr_data):
    shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [25]:
import os
import cv2
import shutil

def get_cropped_image_if_2_eyes(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Unable to read image at {image_path}")
            return None
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                return roi_color
        return None
    except Exception as e:
        print(f"Error processing image at {image_path}: {e}")
        return None

# Load Haar cascades
face_cascade = cv2.CascadeClassifier('model/opencv/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('model/opencv/haarcascade_eye.xml')

# Paths
path_to_data = 'model/dataset'
path_to_cr_data = 'model/dataset/cropped'

# Remove existing cropped images directory
if os.path.exists(path_to_cr_data):
    shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

# List to store the paths of all subdirectories
img_dirs = [entry.path for entry in os.scandir(path_to_data) if entry.is_dir()]

cropped_image_dirs = []
celebrity_file_names_dict = {}

for img_dir in img_dirs:
    celebrity_name = os.path.basename(img_dir)
    print(f"Processing celebrity: {celebrity_name}")
    
    celebrity_file_names_dict[celebrity_name] = []
    
    for entry in os.scandir(img_dir):
        if entry.is_file() and entry.name.lower().endswith(('.jpg', '.jpeg', '.png')):
            cropped_image = get_cropped_image_if_2_eyes(entry.path)
            if cropped_image is not None:
                cropped_folder = os.path.join(path_to_cr_data, celebrity_name)
                if not os.path.exists(cropped_folder):
                    os.makedirs(cropped_folder)
                    cropped_image_dirs.append(cropped_folder)
                    print("Generating cropped images in folder:", cropped_folder)
                
                cropped_file_name = f"{celebrity_name}_{len(celebrity_file_names_dict[celebrity_name]) + 1}.png"
                cropped_file_path = os.path.join(cropped_folder, cropped_file_name)
                
                cv2.imwrite(cropped_file_path, cropped_image)
                celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
        else:
            print(f"Ignoring non-image file: {entry.path}")


In [26]:
celebrity_file_names_dict = {}
for img_dir in cropped_image_dirs:
    celebrity_name = img_dir.split('/')[-1]
    file_list = []
    for entry in os.scandir(img_dir):
        file_list.append(entry.path)
    
    celebrity_file_names_dict[celebrity_name] = file_list

print(celebrity_file_names_dict)


In [27]:
class_dict={}
count=0
for celebrity_name in celebrity_file_names_dict.keys():
    class_dict[celebrity_name]=count
    count=count+1
class_dict

In [28]:
X , y = [] , []

for celebrity_name , training_files in celebrity_file_names_dict.items():
    
    for training_image in training_files:
        img=cv2.imread(training_image)
        if img is None:
            continue
        scalled_raw_img=cv2.resize(img,(32,32))
        img_har=w2d(img , 'db1' , 5)
        scalled_img_har=cv2.resize(img_har,(32,32))
        combined_img=np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
        X.append(combined_img)
        y.append(class_dict[celebrity_name])

        
        

In [29]:
len(X[0])

In [30]:
32*32*3 + 32*32

In [31]:
X[0]

In [32]:
y[0]

In [33]:
X=np.array(X).reshape(len(X),4096).astype(float)
X.shape

Data cleaning process is done. Now we are ready to train our model

We will use SVM with rbf kernel tuned with heuristic finetuning

In [34]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [35]:
X_train , X_test , y_train , y_test = train_test_split(X , y , random_state=0)

pipe=Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel='rbf' , C=10))])

pipe.fit(X_train , y_train)
pipe.score(X_test , y_test)

In [36]:
print(classification_report(y_test , pipe.predict(X_test)))

Let's use GridSearch to try out different models with different parameters. Goal is to come up with best model with best fined tuned parameters.

In [37]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [38]:
model_params={
    'svm' :{
        'model': svm.SVC(gamma='auto' , probability=True) , 
        'params' : {
            'svc__C' :[1,10,100,1000] ,
            'svc__kernel' : ['rbf' , 'linear']
        }
    } , 
    'random_forest' :{
        'model' : RandomForestClassifier() , 
        'params' : {
            'randomforestclassifier__n_estimators' :[1,5,10]
        }
    } , 
    'logistic_regression' :{
        'model' : LogisticRegression(solver='liblinear' , multi_class='auto') , 
        'params' : {
            'logisticregression__C' : [1,5,10]
            
        }
    }
}

In [39]:
scores=[]
best_estimators={}
import pandas as pd
for algo , mp in model_params.items():
    pipe=make_pipeline(StandardScaler() , mp['model'])
    clf=GridSearchCV(pipe , mp['params'] , cv= 5 , return_train_score=False , n_jobs=-1, verbose=1)
    clf.fit(X_train , y_train)
    scores.append({
        'model' : algo , 
        'best_score' : clf.best_score_ ,
        'best_params' : clf.best_params_
    })
    best_estimators[algo]=clf.best_estimator_
    
df=pd.DataFrame(scores , columns=['model' , 'best_score' , 'best_params'])
df

In [40]:
best_estimators

In [41]:
best_estimators['svm'].score(X_test , y_test)

In [42]:
best_estimators['random_forest'].score(X_test , y_test)

In [43]:
best_estimators['logistic_regression'].score(X_test , y_test)

In [44]:
best_clf=best_estimators['svm']

In [45]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test , best_clf.predict(X_test))
cm

In [46]:
import seaborn as sns
plt.figure(figsize=(10,7))
sns.heatmap(cm , annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [47]:
class_dict

Save the trained model

In [48]:
import joblib as jb
#Save the model as a pickle in a file
jb.dump(best_clf , "saved_model.pkl")

Save class dictionary

In [49]:
import json
with open("class_dictionary.json" , "w") as f:
    f.write(json.dumps(class_dict))